# Sales Forecasting Model

## Objective
Build predictive models for retail sales forecasting using:
- Traditional statistical methods (ARIMA)
- Machine Learning (Random Forest, XGBoost)
- Time series specific models (Prophet)

Dataset: https://www.kaggle.com/datasets/mehmettahiraslan/customer-shopping-dataset


In [ ]:
# Import libraries
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root / 'scripts'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Try to import XGBoost
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("XGBoost not available. Install with: pip install xgboost")

# Try to import Prophet
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    PROPHET_AVAILABLE = False
    print("Prophet not available. Install with: pip install prophet")

from utils.db_operations import MongoDBHandler
from etl.feature_engineering import FeatureEngineer

print("✓ Libraries imported successfully")


In [ ]:
# Load transformed data with features
db_handler = MongoDBHandler()
df = db_handler.read_to_dataframe('transformed_sales')

print(f"Dataset shape: {df.shape}")
df.head()


In [ ]:
# Prepare data for modeling
# Select features and target
feature_cols = [col for col in df.columns if col not in ['invoice_no', 'customer_id', 'invoice_date', 'total_amount']]
X = df[feature_cols].select_dtypes(include=[np.number])
y = df['total_amount']

print(f"Original data shape: {df.shape}")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Check for missing values
print(f"\nMissing values in features: {X.isnull().sum().sum()}")
print(f"Missing values in target: {y.isnull().sum()}")

# Check which columns have missing values
missing_cols = X.columns[X.isnull().any()].tolist()
print(f"\nColumns with missing values: {missing_cols}")

# Show sample of missing values
if missing_cols:
    print(f"\nSample of missing values in first few columns:")
    for col in missing_cols[:5]:
        missing_count = X[col].isnull().sum()
        print(f"{col}: {missing_count} missing values")

# Handle missing values with a more robust strategy
print(f"\nHandling missing values...")

# For columns that are all NaN, fill with 0
# For other columns, fill with median
X_filled = X.copy()
for col in X_filled.columns:
    if X_filled[col].isnull().all():
        print(f"Column '{col}' is all NaN, filling with 0")
        X_filled[col] = 0
    else:
        median_val = X_filled[col].median()
        if pd.isna(median_val):
            print(f"Column '{col}' median is NaN, filling with 0")
            X_filled[col] = X_filled[col].fillna(0)
        else:
            X_filled[col] = X_filled[col].fillna(median_val)

# Handle target variable
y_filled = y.fillna(y.median())

print(f"\nAfter filling missing values:")
print(f"Features shape: {X_filled.shape}")
print(f"Target shape: {y_filled.shape}")
print(f"Missing values in features: {X_filled.isnull().sum().sum()}")
print(f"Missing values in target: {y_filled.isnull().sum()}")

# Verify no NaN values remain
if X_filled.isnull().sum().sum() > 0:
    print(f"\nWarning: Still have {X_filled.isnull().sum().sum()} missing values!")
    print("Columns with remaining NaN values:")
    nan_cols = X_filled.columns[X_filled.isnull().any()].tolist()
    for col in nan_cols:
        print(f"  {col}: {X_filled[col].isnull().sum()} missing")
    # Fill any remaining NaN with 0
    X_filled = X_filled.fillna(0)
    print("Filled remaining NaN values with 0")

# Note: Train-test split will be done after data quality check


In [ ]:
# Data Quality Check and Feature Selection
print("=== DATA QUALITY CHECK ===")

# Check for constant features (zero variance)
constant_features = []
for col in X_filled.columns:
    if X_filled[col].nunique() <= 1:
        constant_features.append(col)
        print(f"Constant feature: {col} (unique values: {X_filled[col].nunique()})")

# Remove constant features
if constant_features:
    print(f"\nRemoving {len(constant_features)} constant features...")
    X_filled = X_filled.drop(columns=constant_features)
    print(f"Features after removing constants: {X_filled.shape[1]}")

# Check for highly correlated features
print(f"\nChecking for highly correlated features...")
correlation_matrix = X_filled.corr().abs()
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)

# Find pairs with correlation > 0.95
high_corr_pairs = []
for i in range(len(upper_triangle.columns)):
    for j in range(i):
        if upper_triangle.iloc[i, j] > 0.95:
            high_corr_pairs.append((upper_triangle.columns[i], upper_triangle.columns[j], upper_triangle.iloc[i, j]))

if high_corr_pairs:
    print(f"Found {len(high_corr_pairs)} highly correlated feature pairs (>0.95):")
    for feat1, feat2, corr in high_corr_pairs[:5]:  # Show first 5
        print(f"  {feat1} <-> {feat2}: {corr:.3f}")

# Check data types and ranges
print(f"\nData Summary:")
print(f"Features shape: {X_filled.shape}")
print(f"Data types: {X_filled.dtypes.value_counts().to_dict()}")
print(f"Value ranges:")
for col in X_filled.columns[:5]:  # Show first 5 columns
    print(f"  {col}: [{X_filled[col].min():.2f}, {X_filled[col].max():.2f}]")

# Final verification
print(f"\nFinal verification:")
print(f"Missing values: {X_filled.isnull().sum().sum()}")
print(f"Infinite values: {np.isinf(X_filled).sum().sum()}")
print(f"Ready for modeling: {X_filled.isnull().sum().sum() == 0 and np.isinf(X_filled).sum().sum() == 0}")

# Train-test split (after data cleaning)
print(f"\n=== TRAIN-TEST SPLIT ===")
X_train, X_test, y_train, y_test = train_test_split(X_filled, y_filled, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Verify no missing values in train/test sets
print(f"Training set missing values: {X_train.isnull().sum().sum()}")
print(f"Test set missing values: {X_test.isnull().sum().sum()}")
print(f"Training target missing values: {y_train.isnull().sum()}")
print(f"Test target missing values: {y_test.isnull().sum()}")


In [ ]:
# Model 1: Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

# Evaluate
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression Results:")
print(f"RMSE: {rmse_lr:.2f}")
print(f"MAE: {mae_lr:.2f}")
print(f"R² Score: {r2_lr:.4f}")


In [ ]:
# Investigate the perfect scores issue
print("=== INVESTIGATING PERFECT SCORES ===")

# Check if there's data leakage
print("1. Checking for data leakage...")
print(f"Target variable name: 'total_amount'")
print(f"Features containing 'total_amount': {[col for col in X_train.columns if 'total_amount' in col.lower()]}")

# Check if target is perfectly predictable
print(f"\n2. Target variable statistics:")
print(f"Target mean: {y_train.mean():.4f}")
print(f"Target std: {y_train.std():.4f}")
print(f"Target min: {y_train.min():.4f}")
print(f"Target max: {y_train.max():.4f}")

# Check for constant target
print(f"Target unique values: {y_train.nunique()}")
print(f"Target is constant: {y_train.nunique() == 1}")

# Check feature-target correlation
print(f"\n3. Feature-target correlations (top 10):")
correlations = []
for col in X_train.columns:
    corr = X_train[col].corr(y_train)
    if not pd.isna(corr):
        correlations.append((col, corr))

correlations.sort(key=lambda x: abs(x[1]), reverse=True)
for col, corr in correlations[:10]:
    print(f"  {col}: {corr:.4f}")

# Check if any feature is identical to target
print(f"\n4. Checking for identical features to target:")
for col in X_train.columns:
    if X_train[col].equals(y_train):
        print(f"  WARNING: Feature '{col}' is identical to target!")
    elif X_train[col].corr(y_train) > 0.999:
        print(f"  WARNING: Feature '{col}' is almost identical to target (corr: {X_train[col].corr(y_train):.4f})")

# Check predictions vs actual
print(f"\n5. Prediction analysis:")
print(f"Predictions range: [{y_pred_lr.min():.4f}, {y_pred_lr.max():.4f}]")
print(f"Actual range: [{y_test.min():.4f}, {y_test.max():.4f}]")
print(f"Predictions == Actual: {np.allclose(y_pred_lr, y_test)}")

# Note: Random Forest model not yet trained, skipping feature importance for now
print(f"\n6. Data Leakage Summary:")
print(f"Found {len([col for col in X_train.columns if X_train[col].corr(y_train) > 0.999])} features with perfect correlation to target")
print(f"These features are essentially identical to the target variable, causing perfect predictions")
print(f"This is why we get RMSE=0.00, MAE=0.00, and R²=1.0000")


In [ ]:
# Fix data leakage by removing problematic features
print("=== FIXING DATA LEAKAGE ===")

# Remove features that are highly correlated with target (potential data leakage)
high_corr_features = []
for col in X_train.columns:
    corr = abs(X_train[col].corr(y_train))
    if corr > 0.95:  # Very high correlation threshold
        high_corr_features.append(col)
        print(f"Removing high correlation feature: {col} (corr: {corr:.4f})")

# Remove features that might contain target information
target_related_features = [col for col in X_train.columns if 'total_amount' in col.lower()]
print(f"\nRemoving target-related features: {target_related_features}")

# Combine all problematic features
problematic_features = list(set(high_corr_features + target_related_features))
print(f"\nTotal problematic features to remove: {len(problematic_features)}")

# Create clean feature set
X_clean = X_train.drop(columns=problematic_features)
X_test_clean = X_test.drop(columns=problematic_features)

print(f"\nOriginal features: {X_train.shape[1]}")
print(f"Clean features: {X_clean.shape[1]}")
print(f"Removed features: {X_train.shape[1] - X_clean.shape[1]}")

# Verify no data leakage in clean data
print(f"\nVerifying clean data:")
clean_correlations = []
for col in X_clean.columns:
    corr = abs(X_clean[col].corr(y_train))
    if corr > 0.9:
        clean_correlations.append((col, corr))

if clean_correlations:
    print(f"WARNING: Still have {len(clean_correlations)} highly correlated features:")
    for col, corr in clean_correlations:
        print(f"  {col}: {corr:.4f}")
else:
    print("✓ No highly correlated features remaining")

# Retrain models with clean data
print("\n=== RETRAINING MODELS WITH CLEAN DATA ===")

# Linear Regression
lr_clean = LinearRegression()
lr_clean.fit(X_clean, y_train)
y_pred_lr_clean = lr_clean.predict(X_test_clean)

# Evaluate clean Linear Regression
mse_lr_clean = mean_squared_error(y_test, y_pred_lr_clean)
rmse_lr_clean = np.sqrt(mse_lr_clean)
mae_lr_clean = mean_absolute_error(y_test, y_pred_lr_clean)
r2_lr_clean = r2_score(y_test, y_pred_lr_clean)

print("Clean Linear Regression Results:")
print(f"RMSE: {rmse_lr_clean:.2f}")
print(f"MAE: {mae_lr_clean:.2f}")
print(f"R² Score: {r2_lr_clean:.4f}")

# Random Forest
rf_clean = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_clean.fit(X_clean, y_train)
y_pred_rf_clean = rf_clean.predict(X_test_clean)

# Evaluate clean Random Forest
mse_rf_clean = mean_squared_error(y_test, y_pred_rf_clean)
rmse_rf_clean = np.sqrt(mse_rf_clean)
mae_rf_clean = mean_absolute_error(y_test, y_pred_rf_clean)
r2_rf_clean = r2_score(y_test, y_pred_rf_clean)

print("\nClean Random Forest Results:")
print(f"RMSE: {rmse_rf_clean:.2f}")
print(f"MAE: {mae_rf_clean:.2f}")
print(f"R² Score: {r2_rf_clean:.4f}")

# Feature importance for clean Random Forest
print("\nClean Random Forest Feature Importance:")
feature_importance_clean = pd.DataFrame({
    'feature': X_clean.columns,
    'importance': rf_clean.feature_importances_
}).sort_values('importance', ascending=False).head(10)

print(feature_importance_clean)


In [ ]:
# Model 2: Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Evaluate
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Results:")
print(f"RMSE: {rmse_rf:.2f}")
print(f"MAE: {mae_rf:.2f}")
print(f"R² Score: {r2_rf:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X_filled.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False).head(10)

print("\nTop 10 Important Features:")
print(feature_importance)


In [ ]:
# Model comparison visualization (using clean results)
print("=== MODEL COMPARISON (CLEAN DATA) ===")

models_data = {
    'Model': ['Linear Regression (Clean)', 'Random Forest (Clean)'],
    'RMSE': [rmse_lr_clean, rmse_rf_clean],
    'MAE': [mae_lr_clean, mae_rf_clean],
    'R² Score': [r2_lr_clean, r2_rf_clean]
}

# Add XGBoost if available
if XGBOOST_AVAILABLE and rmse_xgb is not None:
    models_data['Model'].append('XGBoost')
    models_data['RMSE'].append(rmse_xgb)
    models_data['MAE'].append(mae_xgb)
    models_data['R² Score'].append(r2_xgb)

results = pd.DataFrame(models_data)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

results.plot(x='Model', y='RMSE', kind='bar', ax=axes[0], legend=False)
axes[0].set_title('RMSE Comparison (Lower is Better)')
axes[0].tick_params(axis='x', rotation=45)

results.plot(x='Model', y='MAE', kind='bar', ax=axes[1], legend=False, color='orange')
axes[1].set_title('MAE Comparison (Lower is Better)')
axes[1].tick_params(axis='x', rotation=45)

results.plot(x='Model', y='R² Score', kind='bar', ax=axes[2], legend=False, color='green')
axes[2].set_title('R² Score Comparison (Higher is Better)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nClean Model Comparison Summary:")
print(results.round(4))

# Compare with original (problematic) results
print("\n=== COMPARISON: ORIGINAL vs CLEAN ===")
comparison_data = {
    'Model': ['Linear Regression (Original)', 'Linear Regression (Clean)', 
              'Random Forest (Original)', 'Random Forest (Clean)'],
    'RMSE': [rmse_lr, rmse_lr_clean, rmse_rf, rmse_rf_clean],
    'MAE': [mae_lr, mae_lr_clean, mae_rf, mae_rf_clean],
    'R² Score': [r2_lr, r2_lr_clean, r2_rf, r2_rf_clean]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.round(4))

print("\n=== ANALYSIS ===")
print("The original models showed perfect scores due to data leakage.")
print("Features containing 'total_amount' or highly correlated with target were removed.")
print("Clean models show more realistic performance metrics.")


In [ ]:
# Model 3: XGBoost (if available)
if XGBOOST_AVAILABLE:
    xgb_model = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1
    )
    xgb_model.fit(X_train, y_train)
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    rmse_xgb = np.sqrt(mse_xgb)
    mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    print("XGBoost Results:")
    print(f"RMSE: {rmse_xgb:.2f}")
    print(f"MAE: {mae_xgb:.2f}")
    print(f"R² Score: {r2_xgb:.4f}")
    
    # Feature importance for XGBoost
    xgb_importance = pd.DataFrame({
        'feature': X_filled.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False).head(10)
    
    print("\nTop 10 Important Features (XGBoost):")
    print(xgb_importance)
else:
    print("XGBoost not available. Skipping XGBoost model.")
    rmse_xgb = mae_xgb = r2_xgb = None


In [ ]:
# Fixed Prophet Time Series Forecasting with Error Handling
print("\n=== PROPHET TIME SERIES FORECASTING (FIXED) ===")

if PROPHET_AVAILABLE:
    try:
        # Prepare data for Prophet
        prophet_data = daily_sales.copy()
        prophet_data.columns = ['ds', 'y']  # Prophet expects 'ds' for dates and 'y' for values
        
        # Split data for training and testing
        train_size = int(len(prophet_data) * 0.8)
        train_data = prophet_data[:train_size]
        test_data = prophet_data[train_size:]
        
        print(f"Training data: {len(train_data)} days")
        print(f"Test data: {len(test_data)} days")
        
        # Try Prophet with different configurations
        try:
            # First try with minimal configuration
            model = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False
            )
            model.fit(train_data)
            
            # Make predictions
            future = model.make_future_dataframe(periods=len(test_data))
            forecast = model.predict(future)
            
            # Evaluate on test set
            predictions = forecast['yhat'][train_size:].values
            actual = test_data['y'].values
            
            mse_prophet = mean_squared_error(actual, predictions)
            rmse_prophet = np.sqrt(mse_prophet)
            mae_prophet = mean_absolute_error(actual, predictions)
            r2_prophet = r2_score(actual, predictions)
            
            print(f"\nProphet Results:")
            print(f"RMSE: {rmse_prophet:.2f}")
            print(f"MAE: {mae_prophet:.2f}")
            print(f"R² Score: {r2_prophet:.4f}")
            
            # Plot Prophet forecast
            fig = model.plot(forecast)
            plt.title('Prophet Time Series Forecast')
            plt.show()
            
            # Plot components
            fig = model.plot_components(forecast)
            plt.show()
            
        except Exception as e:
            print(f"Prophet failed: {str(e)}")
            print("Using alternative time series approach...")
            
            # Alternative: Simple time series forecasting using sklearn
            from sklearn.linear_model import LinearRegression
            from sklearn.preprocessing import PolynomialFeatures
            
            # Create time-based features
            train_data_alt = train_data.copy()
            train_data_alt['day_of_year'] = pd.to_datetime(train_data_alt['ds']).dt.dayofyear
            train_data_alt['month'] = pd.to_datetime(train_data_alt['ds']).dt.month
            train_data_alt['day_of_week'] = pd.to_datetime(train_data_alt['ds']).dt.dayofweek
            train_data_alt['quarter'] = pd.to_datetime(train_data_alt['ds']).dt.quarter
            
            test_data_alt = test_data.copy()
            test_data_alt['day_of_year'] = pd.to_datetime(test_data_alt['ds']).dt.dayofyear
            test_data_alt['month'] = pd.to_datetime(test_data_alt['ds']).dt.month
            test_data_alt['day_of_week'] = pd.to_datetime(test_data_alt['ds']).dt.dayofweek
            test_data_alt['quarter'] = pd.to_datetime(test_data_alt['ds']).dt.quarter
            
            # Prepare features
            X_train_ts = train_data_alt[['day_of_year', 'month', 'day_of_week', 'quarter']].values
            y_train_ts = train_data_alt['y'].values
            X_test_ts = test_data_alt[['day_of_year', 'month', 'day_of_week', 'quarter']].values
            y_test_ts = test_data_alt['y'].values
            
            # Fit polynomial regression for time series
            poly = PolynomialFeatures(degree=2, include_bias=False)
            X_train_poly = poly.fit_transform(X_train_ts)
            X_test_poly = poly.transform(X_test_ts)
            
            ts_model = LinearRegression()
            ts_model.fit(X_train_poly, y_train_ts)
            predictions = ts_model.predict(X_test_poly)
            
            # Evaluate
            mse_prophet = mean_squared_error(y_test_ts, predictions)
            rmse_prophet = np.sqrt(mse_prophet)
            mae_prophet = mean_absolute_error(y_test_ts, predictions)
            r2_prophet = r2_score(y_test_ts, predictions)
            
            print(f"\nAlternative Time Series Results (Polynomial Regression):")
            print(f"RMSE: {rmse_prophet:.2f}")
            print(f"MAE: {mae_prophet:.2f}")
            print(f"R² Score: {r2_prophet:.4f}")
            
            # Plot results
            plt.figure(figsize=(15, 6))
            plt.plot(range(len(y_test_ts)), y_test_ts, label='Actual', alpha=0.7, linewidth=2)
            plt.plot(range(len(predictions)), predictions, label='Predicted', alpha=0.7, linewidth=2)
            plt.title('Time Series Forecasting (Alternative Method)')
            plt.xlabel('Time')
            plt.ylabel('Sales Amount')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
            
    except Exception as e:
        print(f"Time series analysis failed: {str(e)}")
        rmse_prophet = mae_prophet = r2_prophet = None
        
else:
    print("Prophet not available. Using alternative time series approach...")
    
    # Alternative time series analysis
    try:
        # Create time-based features for daily sales
        daily_sales_alt = daily_sales.copy()
        daily_sales_alt['day_of_year'] = pd.to_datetime(daily_sales_alt['invoice_date']).dt.dayofyear
        daily_sales_alt['month'] = pd.to_datetime(daily_sales_alt['invoice_date']).dt.month
        daily_sales_alt['day_of_week'] = pd.to_datetime(daily_sales_alt['invoice_date']).dt.dayofweek
        daily_sales_alt['quarter'] = pd.to_datetime(daily_sales_alt['invoice_date']).dt.quarter
        
        # Split data
        train_size = int(len(daily_sales_alt) * 0.8)
        train_data = daily_sales_alt[:train_size]
        test_data = daily_sales_alt[train_size:]
        
        # Prepare features
        X_train_ts = train_data[['day_of_year', 'month', 'day_of_week', 'quarter']].values
        y_train_ts = train_data['total_amount'].values
        X_test_ts = test_data[['day_of_year', 'month', 'day_of_week', 'quarter']].values
        y_test_ts = test_data['total_amount'].values
        
        # Fit polynomial regression
        poly = PolynomialFeatures(degree=2, include_bias=False)
        X_train_poly = poly.fit_transform(X_train_ts)
        X_test_poly = poly.transform(X_test_ts)
        
        ts_model = LinearRegression()
        ts_model.fit(X_train_poly, y_train_ts)
        predictions = ts_model.predict(X_test_poly)
        
        # Evaluate
        mse_prophet = mean_squared_error(y_test_ts, predictions)
        rmse_prophet = np.sqrt(mse_prophet)
        mae_prophet = mean_absolute_error(y_test_ts, predictions)
        r2_prophet = r2_score(y_test_ts, predictions)
        
        print(f"\nAlternative Time Series Results:")
        print(f"RMSE: {rmse_prophet:.2f}")
        print(f"MAE: {mae_prophet:.2f}")
        print(f"R² Score: {r2_prophet:.4f}")
        
        # Plot results
        plt.figure(figsize=(15, 6))
        plt.plot(range(len(y_test_ts)), y_test_ts, label='Actual', alpha=0.7, linewidth=2)
        plt.plot(range(len(predictions)), predictions, label='Predicted', alpha=0.7, linewidth=2)
        plt.title('Time Series Forecasting (Alternative Method)')
        plt.xlabel('Time')
        plt.ylabel('Sales Amount')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
    except Exception as e:
        print(f"Alternative time series failed: {str(e)}")
        rmse_prophet = mae_prophet = r2_prophet = None


In [ ]:
# Quick Fix: Skip Prophet and use alternative time series approach
print("=== TIME SERIES FORECASTING (ALTERNATIVE METHOD) ===")

# Create time series data
df_ts = df.copy()
df_ts['invoice_date'] = pd.to_datetime(df_ts['invoice_date'])

# Create daily sales aggregation
daily_sales = df_ts.groupby('invoice_date')['total_amount'].sum().reset_index()
daily_sales = daily_sales.sort_values('invoice_date')

print(f"Time series data shape: {daily_sales.shape}")
print(f"Date range: {daily_sales['invoice_date'].min()} to {daily_sales['invoice_date'].max()}")

# Create time-based features
daily_sales['day_of_year'] = daily_sales['invoice_date'].dt.dayofyear
daily_sales['month'] = daily_sales['invoice_date'].dt.month
daily_sales['day_of_week'] = daily_sales['invoice_date'].dt.dayofweek
daily_sales['quarter'] = daily_sales['invoice_date'].dt.quarter
daily_sales['year'] = daily_sales['invoice_date'].dt.year

# Split data for training and testing
train_size = int(len(daily_sales) * 0.8)
train_data = daily_sales[:train_size]
test_data = daily_sales[train_size:]

print(f"Training data: {len(train_data)} days")
print(f"Test data: {len(test_data)} days")

# Prepare features
X_train_ts = train_data[['day_of_year', 'month', 'day_of_week', 'quarter', 'year']].values
y_train_ts = train_data['total_amount'].values
X_test_ts = test_data[['day_of_year', 'month', 'day_of_week', 'quarter', 'year']].values
y_test_ts = test_data['total_amount'].values

# Fit polynomial regression for time series
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_ts)
X_test_poly = poly.transform(X_test_ts)

ts_model = LinearRegression()
ts_model.fit(X_train_poly, y_train_ts)
predictions = ts_model.predict(X_test_poly)

# Evaluate
mse_prophet = mean_squared_error(y_test_ts, predictions)
rmse_prophet = np.sqrt(mse_prophet)
mae_prophet = mean_absolute_error(y_test_ts, predictions)
r2_prophet = r2_score(y_test_ts, predictions)

print(f"\nTime Series Forecasting Results:")
print(f"RMSE: {rmse_prophet:.2f}")
print(f"MAE: {mae_prophet:.2f}")
print(f"R² Score: {r2_prophet:.4f}")

# Plot results
plt.figure(figsize=(15, 8))

# Plot 1: Time series forecast
plt.subplot(2, 1, 1)
plt.plot(range(len(y_test_ts)), y_test_ts, label='Actual', alpha=0.7, linewidth=2, color='blue')
plt.plot(range(len(predictions)), predictions, label='Predicted', alpha=0.7, linewidth=2, color='red')
plt.title('Time Series Forecasting: Actual vs Predicted')
plt.xlabel('Time (Days)')
plt.ylabel('Sales Amount')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Scatter plot
plt.subplot(2, 1, 2)
plt.scatter(y_test_ts, predictions, alpha=0.6, color='green')
plt.plot([y_test_ts.min(), y_test_ts.max()], [y_test_ts.min(), y_test_ts.max()], 'r--', lw=2)
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Prediction Accuracy Scatter Plot')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Feature importance for time series
feature_names = ['day_of_year', 'month', 'day_of_week', 'quarter', 'year']
feature_importance_ts = pd.DataFrame({
    'feature': feature_names,
    'coefficient': ts_model.coef_[:len(feature_names)]
}).sort_values('coefficient', key=abs, ascending=False)

print("\nTime Series Feature Importance:")
print(feature_importance_ts)


In [ ]:
# Model comparison visualization
models_data = {
    'Model': ['Linear Regression', 'Random Forest'],
    'RMSE': [rmse_lr, rmse_rf],
    'MAE': [mae_lr, mae_rf],
    'R² Score': [r2_lr, r2_rf]
}

# Add XGBoost if available
if XGBOOST_AVAILABLE and rmse_xgb is not None:
    models_data['Model'].append('XGBoost')
    models_data['RMSE'].append(rmse_xgb)
    models_data['MAE'].append(mae_xgb)
    models_data['R² Score'].append(r2_xgb)

results = pd.DataFrame(models_data)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

results.plot(x='Model', y='RMSE', kind='bar', ax=axes[0], legend=False)
axes[0].set_title('RMSE Comparison')
axes[0].tick_params(axis='x', rotation=45)

results.plot(x='Model', y='MAE', kind='bar', ax=axes[1], legend=False, color='orange')
axes[1].set_title('MAE Comparison')
axes[1].tick_params(axis='x', rotation=45)

results.plot(x='Model', y='R² Score', kind='bar', ax=axes[2], legend=False, color='green')
axes[2].set_title('R² Score Comparison')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nModel Comparison Summary:")
print(results.round(4))


In [ ]:
# Time Series Analysis
print("=== TIME SERIES ANALYSIS ===")

# Convert invoice_date to datetime and create time series
df_ts = df.copy()
df_ts['invoice_date'] = pd.to_datetime(df_ts['invoice_date'])

# Create daily sales aggregation
daily_sales = df_ts.groupby('invoice_date')['total_amount'].sum().reset_index()
daily_sales = daily_sales.sort_values('invoice_date')

print(f"Time series data shape: {daily_sales.shape}")
print(f"Date range: {daily_sales['invoice_date'].min()} to {daily_sales['invoice_date'].max()}")

# Plot time series
plt.figure(figsize=(15, 6))
plt.plot(daily_sales['invoice_date'], daily_sales['total_amount'])
plt.title('Daily Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales Amount')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Basic time series statistics
print(f"\nTime Series Statistics:")
print(f"Mean daily sales: {daily_sales['total_amount'].mean():.2f}")
print(f"Std daily sales: {daily_sales['total_amount'].std():.2f}")
print(f"Min daily sales: {daily_sales['total_amount'].min():.2f}")
print(f"Max daily sales: {daily_sales['total_amount'].max():.2f}")


In [ ]:
# Prophet Time Series Forecasting (if available)
if PROPHET_AVAILABLE:
    print("\n=== PROPHET TIME SERIES FORECASTING ===")
    
    # Prepare data for Prophet
    prophet_data = daily_sales.copy()
    prophet_data.columns = ['ds', 'y']  # Prophet expects 'ds' for dates and 'y' for values
    
    # Split data for training and testing
    train_size = int(len(prophet_data) * 0.8)
    train_data = prophet_data[:train_size]
    test_data = prophet_data[train_size:]
    
    print(f"Training data: {len(train_data)} days")
    print(f"Test data: {len(test_data)} days")
    
    # Create and fit Prophet model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='multiplicative'
    )
    
    model.fit(train_data)
    
    # Make predictions
    future = model.make_future_dataframe(periods=len(test_data))
    forecast = model.predict(future)
    
    # Evaluate on test set
    predictions = forecast['yhat'][train_size:].values
    actual = test_data['y'].values
    
    mse_prophet = mean_squared_error(actual, predictions)
    rmse_prophet = np.sqrt(mse_prophet)
    mae_prophet = mean_absolute_error(actual, predictions)
    r2_prophet = r2_score(actual, predictions)
    
    print(f"\nProphet Results:")
    print(f"RMSE: {rmse_prophet:.2f}")
    print(f"MAE: {mae_prophet:.2f}")
    print(f"R² Score: {r2_prophet:.4f}")
    
    # Plot Prophet forecast
    fig = model.plot(forecast)
    plt.title('Prophet Time Series Forecast')
    plt.show()
    
    # Plot components
    fig = model.plot_components(forecast)
    plt.show()
    
else:
    print("\nProphet not available. Skipping time series forecasting.")
    rmse_prophet = mae_prophet = r2_prophet = None


In [ ]:
# Final Model Comparison
print("\n=== FINAL MODEL COMPARISON ===")

# Create comprehensive comparison
all_models_data = {
    'Model': ['Linear Regression', 'Random Forest'],
    'RMSE': [rmse_lr, rmse_rf],
    'MAE': [mae_lr, mae_rf],
    'R² Score': [r2_lr, r2_rf],
    'Type': ['ML', 'ML']
}

# Add XGBoost if available
if XGBOOST_AVAILABLE and rmse_xgb is not None:
    all_models_data['Model'].append('XGBoost')
    all_models_data['RMSE'].append(rmse_xgb)
    all_models_data['MAE'].append(mae_xgb)
    all_models_data['R² Score'].append(r2_xgb)
    all_models_data['Type'].append('ML')

# Add Prophet if available
if PROPHET_AVAILABLE and rmse_prophet is not None:
    all_models_data['Model'].append('Prophet')
    all_models_data['RMSE'].append(rmse_prophet)
    all_models_data['MAE'].append(mae_prophet)
    all_models_data['R² Score'].append(r2_prophet)
    all_models_data['Type'].append('Time Series')

final_results = pd.DataFrame(all_models_data)

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# RMSE comparison
final_results.plot(x='Model', y='RMSE', kind='bar', ax=axes[0,0], legend=False, color='skyblue')
axes[0,0].set_title('RMSE Comparison (Lower is Better)')
axes[0,0].tick_params(axis='x', rotation=45)

# MAE comparison
final_results.plot(x='Model', y='MAE', kind='bar', ax=axes[0,1], legend=False, color='lightcoral')
axes[0,1].set_title('MAE Comparison (Lower is Better)')
axes[0,1].tick_params(axis='x', rotation=45)

# R² Score comparison
final_results.plot(x='Model', y='R² Score', kind='bar', ax=axes[1,0], legend=False, color='lightgreen')
axes[1,0].set_title('R² Score Comparison (Higher is Better)')
axes[1,0].tick_params(axis='x', rotation=45)

# Model type distribution
type_counts = final_results['Type'].value_counts()
axes[1,1].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', startangle=90)
axes[1,1].set_title('Model Type Distribution')

plt.tight_layout()
plt.show()

print("\nFinal Model Comparison Summary:")
print(final_results.round(4))

# Find best model
best_rmse_idx = final_results['RMSE'].idxmin()
best_r2_idx = final_results['R² Score'].idxmax()

print(f"\nBest Model by RMSE: {final_results.loc[best_rmse_idx, 'Model']} (RMSE: {final_results.loc[best_rmse_idx, 'RMSE']:.4f})")
print(f"Best Model by R² Score: {final_results.loc[best_r2_idx, 'Model']} (R²: {final_results.loc[best_r2_idx, 'R² Score']:.4f})")
